# Project Face Recognition

1.Collect data of various persons
  - Ask multiple people to come in front of the web cam and click 20 pictures of each individual
  - Store the part of the image containing the image (haarcascade to detect the face)

2.Train a classifier to learn who the person is (classification)
  -Load the training Data (.npy arrays)
  -Store the data and target values (labels)

3.Predicting the name of the person
  -Read the video stream
  -Extract the face out of it
  -predict the label for that face
       -logistic regression (parametric algorithm)
       -neural network
       -K-NN (non-parametric: look for similarity in nearest neighbours)

Object Detection(using haarcascade classifier)

In [ ]:
# Read an image or video from WebCam using OpenCv
#Face detection in video
#Click 20 pictures of the person who comes in front of the camera and save them as numpy

import cv2
import numpy as np

# Create a camera object
cam=cv2.VideoCapture(0)

#Ask the name
fileName=input("Enter the name of the person:")
dataset_path=r"C:\Users\abhis\Documents\data_face\data\\"
offset=20

#model
model=cv2.CascadeClassifier(r"C:\Users\abhis\Downloads\raw.githubusercontent.com_avelino_python-opencv-detect_master_haarcascade_frontalface_alt.xml")

#Create a list to save face data
faceData=[]
skip=0

#Read image from camera object
while True:
    success,img=cam.read()
    if not success:
        print("Camera reading failed")
    
    #Store the gray image, on the UI only the colored images will be shown
    grayImg=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    faces=model.detectMultiScale(img,1.3,5)
    #pick the face with largest bounding box
    faces=sorted(faces,key=lambda f:f[2]*f[3]) #Here this represents the area w*h
    #pick the largest face
    if len(faces)>0:  #this condition is imposed so that the program would not crash if there are no elements in the array i.e. if the person face moves sideways no face is detected
        f=faces[-1] #since it is sorted so the last image will have the largest bounding box

        x,y,w,h=f
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    
        #crop and save the largest face
        cropped_face=img[y-offset:y+h+offset,x-offset:x+w+offset]  #slicing operation, y is written first because row is mentioned first and then column
        cropped_face=cv2.resize(cropped_face,(100,100))
        skip+=1
        
        if skip%10==0:
            faceData.append(cropped_face)
            print("Saved so far " + str(len(faceData)))
    
    
    cv2.imshow("Image Window",img)
#     cv2.imshow("Cropped Face",cropped_face)
    
    key=cv2.waitKey(1)  #Pause here for 1ms before you read the next image
    if key==ord('q'):
        break
        
#Write the faceData on the Disk so that we reuse it later
faceData=np.asarray(faceData)
# print(faceData.shape)
m=faceData.shape[0]  #number of pictures clicked
faceData=faceData.reshape((m,-1))    #13,100,100,3=m,-1   so -1 will automatically get updated to the value 100*100*3
print(faceData.shape)
#Save on the Disk as np array
filepath=dataset_path+fileName+".npy"
np.save(filepath,faceData)
print("Data Saved Successfully"+filepath)

#Release camera and destroy window
cam.release()
cv2.destroyAllWindows()

In [1]:
import cv2
import numpy as np
import os

#Data Preparation
dataset_path=r"C:\Users\abhis\Documents\data_face\\"
faceData=[]
labels=[]
nameMap={}

classId=0

for f in os.listdir(dataset_path):
#     print(f)
    if f.endswith(".npy"):
        nameMap[classId]=f[34:-4]
        #X-value
        dataItem=np.load(dataset_path+f)
        m=dataItem.shape[0]
        faceData.append(dataItem)
#         print(dataItem.shape)       
        
        #Y-values
        target=classId*np.ones((m,))
        classId+=1
        labels.append(target)
        
# print(faceData)
# print(labels)

XT=np.concatenate(faceData,axis=0)
yT=np.concatenate(labels,axis=0).reshape((-1,1))  #instead of having it as a vector we can have a column vector

# print(XT.shape)
# print(yT.shape)
# print(nameMap)

#Algorithm
def dist(p,q):
    return np.sqrt(np.sum((p-q)**2))

def knn(X,y,xt,k=5):
    m=X.shape[0]
    dlist=[]
    
    for i in range(m):
        d=dist(X[i],xt)
        dlist.append((d,y[i]))
        
    dlist=sorted(dlist)
    dlist=np.array(dlist[:k])
    labels=dlist[:,-1]
    
    labels,cnts=np.unique(labels,return_counts=True)
    idx=cnts.argmax()
    pred=labels[idx]
    
    return int(pred)

#Predictions
# Create a camera object
cam=cv2.VideoCapture(0)
model=cv2.CascadeClassifier(r"C:\Users\abhis\Downloads\raw.githubusercontent.com_avelino_python-opencv-detect_master_haarcascade_frontalface_alt.xml")

while True:
    success,img=cam.read()
    if not success:
        print("Camera reading failed")  
    
    faces=model.detectMultiScale(img,1.3,5)
    
    #render a box around each face and predict its name
    for f in faces:

        x,y,w,h=f
    
        #crop and save the largest face
        cropped_face=img[y-offset:y+h+offset,x-offset:x+w+offset]  #slicing operation, y is written first because row is mentioned first and then column
        cropped_face=cv2.resize(cropped_face,(100,100))   #openCv resize(only 2 coordinates) is different from numpy reshape(3 coordinates i.e. has color channels also) 
    
        #cv2.imshow("Image Window",img)
        #Predict the name using KNN
        classPredicted=knn(XT,yT,cropped_face.flatten())
        #Name
        namePredicted=nameMap[classPredicted]
        #Display the name and box
        cv2.putText(img,namePredicted,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(0,200,0),2,cv2.LINE_AA)
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
        
    cv2.imshow("Prediction Window",img)
    
    key=cv2.waitKey(1)  #Pause here for 1ms before you read the next image
    if key==ord('q'):
        break
        
cam.release()
cv2.destroyAllWindows()

NameError: name 'offset' is not defined